In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv("diabetes.csv")
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


### 1. Select the following 4 attributes (3 features + 1 class label) :
Glucose, BloodPressure, Insulin, Outcome

In [3]:
X = df[["Glucose", "BloodPressure", "Insulin", "Outcome"]]
X

,Glucose,BloodPressure,Insulin,Outcome
0,148,72,0,1
1,85,66,0,0
2,183,64,0,1
3,89,66,94,0
4,137,40,168,1
...,...,...,...,...
763,101,76,180,0
764,122,70,0,0
765,121,72,112,0
766,126,60,0,1


### 2. Normalize Glucose, BloodPressure and Insulin to [0, 1] using MinMax.

In [4]:
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_scaled

,Glucose,BloodPressure,Insulin,Outcome
0,0.743719,0.590164,0.000000,1.0
1,0.427136,0.540984,0.000000,0.0
2,0.919598,0.524590,0.000000,1.0
3,0.447236,0.540984,0.111111,0.0
4,0.688442,0.327869,0.198582,1.0
...,...,...,...,...
763,0.507538,0.622951,0.212766,0.0
764,0.613065,0.573770,0.000000,0.0
765,0.608040,0.590164,0.132388,0.0
766,0.633166,0.491803,0.000000,1.0


### 3. Store the new data (3 normalized features + 1 class label) in another dataset S.

In [5]:
S = X_scaled

### 4. Modify the MQTT example to do the following:
    The publisher publishes records in S continuously. When it reaches the end of S, it continues to send from the beginning again.
    
    The subscriber continuously receives the data. For each latest record(r) received, apply the 3NN classification to the last 5 records before r, and compare the classification result with the Outcome label in r.
    
        Repeat this for 1000 times, and report the number of correct classifications. 


In [6]:
import time
import paho.mqtt.client as mqtt

# We create a client as the data publisher
mqttc = mqtt.Client()
# We are connecting to port 1883 and set *keepalive* parameter to 60; *keepalive* is the maximum waiting time in seconds.
mqttc.connect("mqtt.eclipseprojects.io", 1883, 60)

# We send the following readings repeatedly. Once we reach the end, we start from the beginning again.
data = S
period = len(data)

if __name__ == "__main__":
    print("Publishing...")
    
    index = 0
    while (index >= 0):
        # Get the current data reading to send out
        row = pd.DataFrame(data.iloc[index % period])
        json = row.to_json()
        
        # Publish the data reading as "myroom_super_S/S"
        mqttc.publish("myroom_super_S/S", json)
        
        # We send the next reading after 2 seconds
        time.sleep(0.1)
        index = index + 1

Publishing...


KeyboardInterrupt: 